### Time Estimate


In [1]:
CONFIG = "./configs/random_10M_2024"
import json
args = None
with open(CONFIG) as f:
    args = json.load(f)
    print(args)
args["gradients_per_file"] = 10000

{'dataset_folder': './curricula/datasets/curriculum_10M_2024', 'raw_dataset_folder_babylm': './train_10M', 'curriculum_path': './curricula/random_10M_2024', 'eval_dataset_folder': './curricula/datasets/curriculum_10M_2024_eval', 'epochs': 10}


In [2]:
total_in_h = 0


In [3]:
from datasets import load_dataset
import datasets
t = lambda x : tokenizer(x["text"], return_special_tokens_mask=True, truncation=True, max_length=512)

dataset = datasets.load_from_disk(args["dataset_folder"])
dataset.set_transform(t)





In [4]:
print(len(dataset) / args["gradients_per_file"], "chunks per checkpoint")

117.9014 chunks per checkpoint


#### gradient extraction

In [5]:
args["num_processes"] = 8
TIME_PER_BATCH_IN_MIN = 597/60

In [6]:
args["num_processes"] = 4
TIME_PER_BATCH_IN_MIN = 350/60

In [7]:
args["num_processes"] = 6
TIME_PER_BATCH_IN_MIN = 450/60

In [8]:
(7.4/(441))*args["num_processes"]

0.10068027210884353

In [9]:
time_if = ((((len(dataset) / args["gradients_per_file"]) *args["epochs"]) /args["num_processes"]) * TIME_PER_BATCH_IN_MIN)/60
print(time_if, "hours total", time_if/args["epochs"], "hours per checkpoint")
total_in_h+=time_if

24.562791666666666 hours total 2.4562791666666666 hours per checkpoint


In [10]:
SIZE_IN_GB_OF_CHUNK = 7.4


In [11]:
print((len(dataset) / args["gradients_per_file"]) * SIZE_IN_GB_OF_CHUNK/1000, "TB disk space required")

0.87247036 TB disk space required


In [12]:
1186/60

19.766666666666666

In [64]:
110/20

5.5

In [66]:
times = [83.8105,
86.6202,
104.5324,
110.7415]

In [71]:
import numpy as np
np.mean([t/20 for t in times])

4.8213075

In [151]:
time_einsum = np.array([14.8114,
14.6099,
14.8382,
14.6639,
10.3062,
10.1424])
np.mean(time_einsum - time_load_single_chunk)/20

0.4114333333333334

#### mean dot

In [161]:
args["batch_size"] = 20
args["num_processes"] = 4

SIZE_IN_GB_OF_CHUNK = 7.4 # ls
time_single_chunk = 15.36 # get from log # 1622/60
time_load_task = 110 # get from log

In [206]:
num_chunks =(((len(dataset) / args["gradients_per_file"])))
num_batches = num_chunks / args["batch_size"]

time_load_single_chunk = 5
import datetime
times = [
(datetime.datetime.strptime("2024-10-30 14:41:58,109", "%Y-%m-%d %H:%M:%S,%f") - datetime.datetime.strptime("2024-10-30 13:47:04,870", "%Y-%m-%d %H:%M:%S,%f")).seconds, 
(datetime.datetime.strptime("2024-10-30 18:24:12,453", "%Y-%m-%d %H:%M:%S,%f") - datetime.datetime.strptime("2024-10-30 17:27:59,086", "%Y-%m-%d %H:%M:%S,%f")).seconds,
(datetime.datetime.strptime("2024-10-30 21:55:07,874", "%Y-%m-%d %H:%M:%S,%f") - datetime.datetime.strptime("2024-10-30 21:03:04,854", "%Y-%m-%d %H:%M:%S,%f")).seconds, 
(datetime.datetime.strptime("2024-10-31 01:42:55,240", "%Y-%m-%d %H:%M:%S,%f") - datetime.datetime.strptime("2024-10-31 00:52:22,333", "%Y-%m-%d %H:%M:%S,%f")).seconds, 
(datetime.datetime.strptime("2024-10-31 05:12:19,552", "%Y-%m-%d %H:%M:%S,%f") - datetime.datetime.strptime("2024-10-31 04:20:36,587", "%Y-%m-%d %H:%M:%S,%f")).seconds]

time_per_checkpoint = np.mean(np.array(times))

time_einsum_per_pair = 0.4114333333333334
time_per_checkpoint

3184.6

In [238]:
import math
math.ceil(num_chunks)

118

In [244]:
from sklearn.utils import gen_even_slices
list(gen_even_slices(math.ceil(num_chunks), 4))

[slice(0, 30, None),
 slice(30, 60, None),
 slice(60, 89, None),
 slice(89, 118, None)]

In [248]:
def batch(lst, k):
        for i in range(0, len(lst), k):
            yield lst[i:i + k]
list(batch([0,1,2,3,4,5,6,7],4))

[[0, 1, 2, 3], [4, 5, 6, 7]]

In [272]:
import math
def batch(lst, batch_size):
        for i in gen_even_slices(len(lst), math.ceil(len(lst)/batch_size)):
            yield lst[i]
list(batch(list(range(math.ceil(num_chunks))),4))

[[0, 1, 2, 3],
 [4, 5, 6, 7],
 [8, 9, 10, 11],
 [12, 13, 14, 15],
 [16, 17, 18, 19],
 [20, 21, 22, 23],
 [24, 25, 26, 27],
 [28, 29, 30, 31],
 [32, 33, 34, 35],
 [36, 37, 38, 39],
 [40, 41, 42, 43],
 [44, 45, 46, 47],
 [48, 49, 50, 51],
 [52, 53, 54, 55],
 [56, 57, 58, 59],
 [60, 61, 62, 63],
 [64, 65, 66, 67],
 [68, 69, 70, 71],
 [72, 73, 74, 75],
 [76, 77, 78, 79],
 [80, 81, 82, 83],
 [84, 85, 86, 87],
 [88, 89, 90, 91],
 [92, 93, 94, 95],
 [96, 97, 98, 99],
 [100, 101, 102, 103],
 [104, 105, 106, 107],
 [108, 109, 110, 111],
 [112, 113, 114],
 [115, 116, 117]]

In [228]:
num_chunks / 29

4.065565517241379

In [208]:
time_einsum_per_pair

0.4114333333333334

In [215]:
num_batches-4

1.8950699999999996

In [216]:
total

8238.770000000082

In [198]:
total/60/60/args["num_processes"]*(1+(10/60))

0.6674929398148215

In [170]:
# args["batch_size"] = 40
# args["num_processes"] = 2

# SIZE_IN_GB_OF_CHUNK = 7.4 # ls
# time_single_chunk = 15.36 # get from log # 1622/60
# time_load_task = 110 # get from log

In [171]:
print("RAM requirements", (args["batch_size"]+1)*args["num_processes"]*SIZE_IN_GB_OF_CHUNK, "GB")

RAM requirements 621.6 GB


In [172]:


num_chunks =(((len(dataset) / args["gradients_per_file"])))
num_batches = num_chunks / args["batch_size"]

time_single_batch = (num_chunks * time_single_chunk) + time_load_task
time_single_epoch = (num_batches * time_single_batch) 
print("Time for a single epoch and one process", time_single_epoch /60/60, "hours")
total_merge = (time_single_epoch * args["epochs"] /args["num_processes"])/60/60
print("Total time with multiprocessing", total_merge, "hours")
total_in_h += total_merge

Time for a single epoch and one process 3.1456183649070217 hours
Total time with multiprocessing 7.864045912267555 hours


## Total

In [16]:
print(total_in_h, "hours")

32.42683757893422 hours
